In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout
import matplotlib.pyplot as plt

2023-12-02 10:29:56.842514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/anyahabana/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
df = pd.read_csv('../Data Preprocessing/Out.csv')
df.tail(10)
df.columns = ['Series', 'Datetime', 'Altitude', 'Humidity', 'Temperature', 'Time',
       'Day', 'Hour_of_Day', 'IsEJeep1', 'IsEJeep2', 'IsEJeep3', 'Latitude',
       'Longitude', 'Lat Diff', 'Long Diff', 'Distance', 'IsStation',
       'IsCharging', 'Cum Distance', 'Next Station Lat', 'Next Station Long',
       'Abs Distance', 'Percent Distance', 'Prev Station Lat',
       'Prev Station Long', 'Prev Abs Distance', 'Prev Cum Distance',
       'Previous Time to Xavier Hall', 'Previous Time to Fine Arts Annex',
       'Previous Time to Loyola House of Studies',
       'Previous Time to Grade School', 'Previous Time to Gate 2.5',
       'Previous Time to Leong Hall', 'Next Time to Xavier Hall',
       'Next Time to Fine Arts Annex', 'Next Time to Loyola House of Studies',
       'Next Time to Grade School', 'Next Time to Gate 2.5',
       'Next Time to Leong Hall', 'Encoded Day', 'Encoded Station',
       'Encoded Prev Station', 'Encoded Next Station']
df.drop(columns=['Series'], inplace=True)

# df['Series'] = df['Series'] + 1
df

,Datetime,Altitude,Humidity,Temperature,Time,Day,Hour_of_Day,IsEJeep1,IsEJeep2,IsEJeep3,...,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall,Encoded Day,Encoded Station,Encoded Prev Station,Encoded Next Station
0,2023-10-23 17:12:39,79.0,82.500000,29.299999,17:12:39,23.0,17.0,1,0,0,...,06:12:33,06:41:23,17:24:22,17:30:29,17:16:30,17:21:38,1,8,8,1
1,2023-10-23 17:12:40,78.7,82.500000,29.299999,17:12:40,23.0,17.0,1,0,0,...,06:12:33,06:41:23,17:24:22,17:30:29,17:34:36,17:21:38,1,0,8,1
2,2023-10-23 17:12:41,78.4,82.500000,29.299999,17:12:41,23.0,17.0,1,0,0,...,06:12:33,06:41:23,17:24:22,17:30:29,17:34:36,17:21:38,1,0,8,1
3,2023-10-23 17:12:42,78.0,82.500000,29.299999,17:12:42,23.0,17.0,1,0,0,...,06:12:33,06:41:23,17:24:22,17:30:29,17:34:36,17:21:38,1,0,8,1
4,2023-10-23 17:12:43,77.7,82.500000,29.299999,17:12:43,23.0,17.0,1,0,0,...,06:12:33,06:41:23,17:24:22,17:30:29,17:34:36,17:21:38,1,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482694,2023-11-12 09:53:23,75.1,80.400002,30.799999,09:53:23,12.0,9.0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,7,5,4,5
482695,2023-11-12 09:53:24,75.5,80.400002,30.799999,09:53:24,12.0,9.0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,7,5,4,5
482696,2023-11-12 09:53:25,75.3,80.400002,30.799999,09:53:25,12.0,9.0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,7,5,4,5
482697,2023-11-12 09:53:26,74.9,80.400002,30.799999,09:53:26,12.0,9.0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,7,5,4,5


In [4]:
# Convert all the times into a timestamp with 0 = 00:00:00 and 86399 = 23:59:59
from datetime import datetime

def time_to_daytimestamp(stamp: str) -> int:
    if stamp is None:
        return -1
    
    if isinstance(stamp, float):
        return -1
    
    h, m, s = stamp.split(':')
    return int(h) * 24 * 60 + int(m) * 60 + int(s)

def datetime_to_daytimestamp(stamp: str) -> int:
    if stamp is None:
        return -1
    
    if isinstance(stamp, float):
        return -1
    
    # Parse the datetime string into a datetime object
    dt_object = datetime.strptime(stamp, "%Y-%m-%d %H:%M:%S")

    # Calculate the total minutes from midnight
    return dt_object.hour * 60 + dt_object.minute

df['Datetime'] = df['Datetime'].apply(lambda time: datetime_to_daytimestamp(time))
df['Time'] = df['Time'].apply(lambda time: time_to_daytimestamp(time))

df['Previous Time to Xavier Hall'] = df['Previous Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Fine Arts Annex'] = df['Previous Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Loyola House of Studies'] = df['Previous Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Grade School'] = df['Previous Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Gate 2.5'] = df['Previous Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Previous Time to Leong Hall'] = df['Previous Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df['Next Time to Xavier Hall'] = df['Next Time to Xavier Hall'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Fine Arts Annex'] = df['Next Time to Fine Arts Annex'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Loyola House of Studies'] = df['Next Time to Loyola House of Studies'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Grade School'] = df['Next Time to Grade School'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Gate 2.5'] = df['Next Time to Gate 2.5'].apply(lambda time: time_to_daytimestamp(time))
df['Next Time to Leong Hall'] = df['Next Time to Leong Hall'].apply(lambda time: time_to_daytimestamp(time))

df

,Datetime,Altitude,Humidity,Temperature,Time,Day,Hour_of_Day,IsEJeep1,IsEJeep2,IsEJeep3,...,Next Time to Xavier Hall,Next Time to Fine Arts Annex,Next Time to Loyola House of Studies,Next Time to Grade School,Next Time to Gate 2.5,Next Time to Leong Hall,Encoded Day,Encoded Station,Encoded Prev Station,Encoded Next Station
0,1032,79.0,82.500000,29.299999,25239,23.0,17.0,1,0,0,...,9393,11123,25942,26309,25470,25778,1,8,8,1
1,1032,78.7,82.500000,29.299999,25240,23.0,17.0,1,0,0,...,9393,11123,25942,26309,26556,25778,1,0,8,1
2,1032,78.4,82.500000,29.299999,25241,23.0,17.0,1,0,0,...,9393,11123,25942,26309,26556,25778,1,0,8,1
3,1032,78.0,82.500000,29.299999,25242,23.0,17.0,1,0,0,...,9393,11123,25942,26309,26556,25778,1,0,8,1
4,1032,77.7,82.500000,29.299999,25243,23.0,17.0,1,0,0,...,9393,11123,25942,26309,26556,25778,1,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482694,593,75.1,80.400002,30.799999,16163,12.0,9.0,0,1,0,...,-1,-1,-1,-1,-1,-1,7,5,4,5
482695,593,75.5,80.400002,30.799999,16164,12.0,9.0,0,1,0,...,-1,-1,-1,-1,-1,-1,7,5,4,5
482696,593,75.3,80.400002,30.799999,16165,12.0,9.0,0,1,0,...,-1,-1,-1,-1,-1,-1,7,5,4,5
482697,593,74.9,80.400002,30.799999,16166,12.0,9.0,0,1,0,...,-1,-1,-1,-1,-1,-1,7,5,4,5


In [5]:
df = df.fillna(-1)

In [6]:
# Assuming 'selected_df' is your DataFrame with timestamps, features, and target columns
# Set the seed for reproducibility
np.random.seed(42)

# Select features and targets
feature_columns = df.columns
target_columns = ['Next Time to Xavier Hall',
       'Next Time to Fine Arts Annex', 'Next Time to Loyola House of Studies',
       'Next Time to Grade School', 'Next Time to Gate 2.5',
       'Next Time to Leong Hall']

# Separate features and targets into different lists
feature_df = df[feature_columns]
target_df = df[target_columns]

In [7]:
# Normalize the data (optional but often beneficial for neural networks)
scaler = StandardScaler()
feature_df[feature_columns] = scaler.fit_transform(feature_df[feature_columns])

In [8]:
# Define sequence length (number of past time steps to consider)
sequence_length = 5

# Prepare sequences for training
feature_sequences = []
target_sequences = []

for i in range(len(df) - sequence_length + 1):
    feature_sequence = feature_df.iloc[i:i+sequence_length]
    target_sequence = target_df.iloc[i:i+sequence_length]
    
    feature_sequences.append(feature_sequence)
    target_sequences.append(target_sequence)

In [ ]:
# Convert sequences to arrays
X = np.array([seq[feature_columns].values for seq in feature_sequences])
y = np.array([seq[target_columns].values for seq in target_sequences])

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Transformer model
def transformer_model(sequence_length, feature_dim, num_heads=2, ff_dim=32, dropout=0.1):
    inputs = Input(shape=(sequence_length, feature_dim))
    
    # Multi-Head Self-Attention
    attention = MultiHeadAttention(num_heads=num_heads)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    # Feedforward Neural Network
    outputs = Dense(ff_dim, activation="relu")(attention)
    outputs = Dense(feature_dim)(outputs)
    outputs = Dropout(dropout)(outputs)
    outputs = LayerNormalization(epsilon=1e-6)(attention + outputs)
    
    # Output layer
    outputs = Dense(len(target_columns), activation="linear")(outputs)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Build and compile the model
model = transformer_model(sequence_length, len(feature_columns))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Plot training loss and validation loss over epochs
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Optionally, inverse transform the scaled predictions and ground truth
y_pred_inv = scaler.inverse_transform(np.concatenate((X_test[:, -1, :len(feature_columns)], y_pred), axis=1))[:, len(feature_columns):]
y_test_inv = scaler.inverse_transform(np.concatenate((X_test[:, -1, :len(feature_columns)], y_test), axis=1))[:, len(feature_columns):]